In [100]:
from bs4 import BeautifulSoup
import requests
import re
import math
import pandas as pd
import numpy as np
import time


response = requests.get("https://sportsbook.draftkings.com//sites/US-NJ-SB/api/v4/eventgroups/88670771/categories/519/subcategories/10061?format=json")
json_data = response.json()
temp = json_data
temp = temp['eventGroup']['offerCategories'][4]['offerSubcategoryDescriptors'][1]['offerSubcategory']['offers'][0][0]#['outcomes']
tourney_round = temp['label']
#print(tourney_round)
outcomes = temp['outcomes']
sportsbook_odds = {}
for line in outcomes:
    team = line['label']
    odds = int(line['oddsAmerican'])
    if odds < 0:
        odds = odds/(odds-100)
    else:
        odds = 1 - odds/(odds+100)
        
    sportsbook_odds.setdefault(team, [None,None])
    sportsbook_odds[team][0] = odds
    
#print(len(outcomes))
# soup = BeautifulSoup(html)
# print(soup)
# table = soup.find('ul', class_='outcomes')#[0].findAll('tr')
# print(table)

#dict_keys(['subcategoryId', 'name', 'offerSubcategory'])

In [101]:
response = requests.get("https://sportsbook.draftkings.com//sites/US-NJ-SB/api/v4/eventgroups/88670771/categories/519?format=json")
json_data = response.json()
temp = json_data
temp = temp['eventGroup']['offerCategories'][4]['offerSubcategoryDescriptors'][0]['offerSubcategory']['offers'][0][0]#['outcomes']
outcomes = temp['outcomes']
tourney_round = temp['label']
for line in outcomes:
    team = line['label']
    odds = int(line['oddsAmerican'])
    if odds < 0:
        odds = odds/(odds-100)
    else:
        odds = 1 - odds/(odds+100)
        
    sportsbook_odds.setdefault(team, [None,None])
    sportsbook_odds[team][1] = odds
sportsbook_odds_df = pd.DataFrame.from_dict(sportsbook_odds, orient = 'index')
# tourney_round = temp['label']
# #print(tourney_round)
# outcomes = temp['outcomes']

In [102]:
sportsbook_odds_df.index.unique()
sportsbook_odds_df.name = sportsbook_odds_df.index
def new_name(name, new_name, df):
    df.loc[df.name == name, 'name'] = new_name#.tolist()
    return df

def combine_teams(name1, name2, combined_name, df):
    team1 = df.loc[df['name'] == name1]
    team2 = df.loc[df['name'] == name2]
    #print(team1)
    #print(team2)
    row_combined = []
    for i in range(2):
        row_combined.append(team1.iloc[0,i]+team2.iloc[0,i])
            
    row_combined.append(combined_name)
    df = df[((df.name != name1) & (df.name != name2))]
    row_combined_series = pd.Series(row_combined, index=df.columns)
    df = df.append(row_combined_series, ignore_index=True)
    return df

#print(sportsbook_odds_df.head())
sportsbook_odds_df['name'] = sportsbook_odds_df.index
#print(sportsbook_odds_df.head())
sportsbook_odds_df = combine_teams('Wyoming', 'Indiana', 'WYO/IU', sportsbook_odds_df)
sportsbook_odds_df = combine_teams('Texas Southern', 'A&M-Corpus Christi', 'TXSO/TCC', sportsbook_odds_df)
sportsbook_odds_df = combine_teams('Notre Dame', 'Notre Dame', 'RUTG/ND', sportsbook_odds_df)
sportsbook_odds_df = combine_teams('Wright State', 'Bryant', 'WRST/BRY', sportsbook_odds_df)

sportsbook_odds_df = new_name('Miami FL', 'Miami', sportsbook_odds_df)
sportsbook_odds_df = new_name('Cal State Fullerton', 'CSU Fullerton', sportsbook_odds_df)
sportsbook_odds_df = new_name('Jacksonville State', 'J\'Ville St', sportsbook_odds_df)
sportsbook_odds_df = new_name('New Mexico St', 'New Mexico State', sportsbook_odds_df)
sportsbook_odds_df = new_name('South Dakota State', 'S Dakota St', sportsbook_odds_df)
sportsbook_odds_df = new_name('Saint Marys', 'Saint Mary\ss', sportsbook_odds_df)
sportsbook_odds_df = new_name('Miami FL', 'Miami', sportsbook_odds_df)

sportsbook_odds_df.head()
sportsbook_odds_df.set_index('name', inplace = True)
print(sportsbook_odds_df.head())
sportsbook_odds_df.columns = ['final_four', 'champs']
# df.columns = ['Leader', 'Time', 'Score']





###### taking pick data from ESPN now
df_picks = pd.read_html('https://fantasy.espn.com/tournament-challenge-bracket/2022/en/whopickedwhom')[0]
#df_picks_fixed = []

picks_dict = {}
for index, row in df_picks.iterrows():
    
    #df_picks_fixed.append([name])
    #print(row)
    for i in range(len(row)):
        name = re.search('\d*([a-zA-Z\s\'\./]*)-.*', row[i]).group(1)
        picks_dict.setdefault(name, [None,None,None,None,None,None])
        
        percentage = re.search('\d+\.\d', row[i]).group(0)
        picks_dict[name][i] = float(percentage)
        
        #df_picks_fixed[index].append(float(percentage))
    
df_picks = pd.DataFrame.from_dict(picks_dict, orient='index', columns = [ '32', '16', '8', '4', '2', '1'])
print(df_picks.head())
sportsbook_odds_df.join(df_picks, lsuffix = '_odds', rsuffix = '_pick percentage')




# RUTG/ND
# WRST/BRY

# ----------------------

# Indiana
# Notre Dame
# New Mexico State
# Wyoming
# A&M-Corpus Christi
# Texas Southern
# Wright State
# Bryant

# import re
# ###### taking pick data from ESPN now
# df_picks = pd.read_html('https://fantasy.espn.com/tournament-challenge-bracket/2022/en/whopickedwhom')[0]
# #df_picks_fixed = []

# picks_dict = {}
# for index, row in df_picks.iterrows():
    
#     #df_picks_fixed.append([name])
#     #print(row)
#     for i in range(len(row)):
#         name = re.search('\d*([a-zA-Z\s\'\./]*)-.*', row[i]).group(1)
#         picks_dict.setdefault(name, [None,None,None,None,None,None])
        
#         percentage = re.search('\d+\.\d', row[i]).group(0)
#         picks_dict[name][i] = float(percentage)
        
#         #df_picks_fixed[index].append(float(percentage))
    
# df_picks = pd.DataFrame.from_dict(picks_dict, orient='index', columns = [ '32', '16', '8', '4', '2', '1'])




# ESPN_teams = df_picks.index.unique().tolist()
# fivethirtyeight_teams = sportsbook_odds_df.index.unique().tolist()
# for name in ESPN_teams:
#     if name not in fivethirtyeight_teams:
#         print(name)

# print('\n----------------------\n')
# for name in fivethirtyeight_teams:
#     if name not in ESPN_teams:
#         print(name)
        

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


                 0         1
name                        
Gonzaga   0.615385  0.250000
Arizona   0.400000  0.133333
Kansas    0.384615  0.105263
Baylor    0.333333  0.076923
Kentucky  0.333333  0.111111
            32    16     8     4     2     1
Kansas    97.0  88.2  65.6  43.7  19.3   8.5
Gonzaga   96.6  90.6  78.8  58.3  42.0  27.8
Arizona   97.0  89.0  68.0  44.1  30.5  13.0
Baylor    96.3  79.3  55.8  30.6  10.4   5.4
Kentucky  95.7  83.4  54.4  30.5  12.6   6.7


,final_four,champs,32,16,8,4,2,1
name,,,,,,,,
Gonzaga,0.615385,0.250000,96.6,90.6,78.8,58.3,42.0,27.8
Arizona,0.400000,0.133333,97.0,89.0,68.0,44.1,30.5,13.0
Kansas,0.384615,0.105263,97.0,88.2,65.6,43.7,19.3,8.5
Baylor,0.333333,0.076923,96.3,79.3,55.8,30.6,10.4,5.4
Kentucky,0.333333,0.111111,95.7,83.4,54.4,30.5,12.6,6.7
...,...,...,...,...,...,...,...,...
J'Ville St,0.003984,0.000999,4.9,2.0,1.0,0.3,0.2,0.1
WYO/IU,0.026230,0.004986,22.9,5.1,1.3,0.5,0.2,0.2
TXSO/TCC,0.008959,0.001998,3.0,1.4,0.8,0.5,0.2,0.1


In [130]:
joined_df = sportsbook_odds_df.join(df_picks, lsuffix = '_odds', rsuffix = '_pick percentage')
final_four_sum = joined_df.final_four.sum()
champs_sum = joined_df.champs.sum()
print(joined_df.head())

joined_df['final_four_leverage_pre_normalized'] = (joined_df.final_four - joined_df.loc[:,'4']/100) * 8
joined_df['champs_leverage_pre_normalized'] = (joined_df.champs - joined_df.loc[:,'1']/100) * 32


joined_df.final_four = joined_df.final_four / (final_four_sum/4)
joined_df.champs = joined_df.champs / (champs_sum)
print(joined_df.head())

joined_df['final_four_leverage'] = (joined_df.final_four - joined_df.loc[:,'4']/100) * 8
joined_df['champs_leverage'] = (joined_df.champs - joined_df.loc[:,'1']/100) * 32
print(joined_df.head())
# (joined_df.champs - joined_df.loc[:,'1']/100) * 32
final_df = joined_df[['final_four_leverage', 'champs_leverage', 'final_four_leverage_pre_normalized', 'champs_leverage_pre_normalized']]
final_df.to_csv('ncaa_tournament_leverage_sportsbook.csv')




          final_four    champs    32    16     8     4     2     1
name                                                              
Gonzaga     0.615385  0.250000  96.6  90.6  78.8  58.3  42.0  27.8
Arizona     0.400000  0.133333  97.0  89.0  68.0  44.1  30.5  13.0
Kansas      0.384615  0.105263  97.0  88.2  65.6  43.7  19.3   8.5
Baylor      0.333333  0.076923  96.3  79.3  55.8  30.6  10.4   5.4
Kentucky    0.333333  0.111111  95.7  83.4  54.4  30.5  12.6   6.7
          final_four    champs    32    16     8     4     2     1  \
name                                                                 
Gonzaga     0.474328  0.180270  96.6  90.6  78.8  58.3  42.0  27.8   
Arizona     0.308314  0.096144  97.0  89.0  68.0  44.1  30.5  13.0   
Kansas      0.296455  0.075903  97.0  88.2  65.6  43.7  19.3   8.5   
Baylor      0.256928  0.055468  96.3  79.3  55.8  30.6  10.4   5.4   
Kentucky    0.256928  0.080120  95.7  83.4  54.4  30.5  12.6   6.7   

          final_four_leverage_pre_normal

In [116]:
joined_df.loc[:,'4']

name
Gonzaga       58.3
Arizona       44.1
Kansas        43.7
Baylor        30.6
Kentucky      30.5
              ... 
J'Ville St     0.3
WYO/IU         0.5
TXSO/TCC       0.5
RUTG/ND        0.4
WRST/BRY       0.5
Name: 4, Length: 62, dtype: float64